# Tuning and training a Convolutional Neural Network for Intrusion Detection
The CNN model will return a value between 0 and 1, which is the probability of the input flow of being malicious. 
You will select a range of hyperparameters to tune the CNN using Random search and you will train it on a dataset of benign traffic and DDoS attack traffic.

The network traffic has been previously pre-processed in a way that packets are grouped in bi-directional traffic flows using the 5-tuple (source IP, destination IP, source Port, destination Port, protocol). Each flow is represented as a 100x20 array, where the rows are the packets of the flow in chronological order, while each column is a packet-level feature in the following order:

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (IAT) | 
| 01 | packet_length (bytes)| 
| 02 | IP_flags_df (0/1) |
| 03 | IP_flags_mf (0/1) |
| 04 | IP_flags_rb (0/1) | 
| 05 | IP_frag_off (0/1) |
| 06 | protocols (integer) |
| 07 | TCP_length (bytes) |
| 08 | TCP_flags_ack (0/1) |
| 09 | TCP_flags_cwr (0/1) |
| 10 | TCP_flags_ece (0/1) |
| 11 | TCP_flags_fin (0/1) |
| 12 | TCP_flags_push (0/1) |
| 13 | TCP_flags_res (0/1) |
| 14 | TCP_flags_reset (0/1) |
| 15 | TCP_flags_syn (0/1) |
| 16 | TCP_flags_urg (0/1) |
| 17 | TCP_window_size (bytes) |
| 18 | UDP_length (bytes) |
| 19 | ICMP_type (code) |

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, GlobalMaxPooling2D, Flatten,MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam,SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
import os
import random as rn
import numpy as np
import logging
import time
from util_functions import *

# disable GPUs for test reproducibility
tf.config.set_visible_devices([], 'GPU')

# print(os.listdir('./Dataset/'))
print(os.listdir('./Reshaped_Dataset/'))

SEED=0

# DATASET_FOLDER = "./Dataset"
DATASET_FOLDER = "./Reshaped_Dataset"
# DATASET_FOLDER = "./Traffic_Manipulator_test/Manipulated_Dataset"

X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5',channels=True)
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5',channels=True)
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5',channels=True)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_val.shape, y_val.shape)

['Dataset_PCA-val.hdf5', 'Dataset_PCA-train.hdf5', 'Dataset_PCA-test.hdf5']
(1446, 2, 2, 1) (1446,)
(179, 2, 2, 1) (179,)
(161, 2, 2, 1) (161,)


In [6]:
def compileModel(model,optimizer='sgd', lr=0.001):
    if optimizer == 'sgd':
        optimizer = SGD(learning_rate=lr, momentum=0.0)
    else:
        optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy']) 

# Model definition
The following method defines the CNN model with configurable hyperparameters. Each hyperparameter has a default value that can be set during the tuning process. In the following cell, your task is to finalise the model by adding a **GlobalMaxPooling2D** layer, a **Flatten** layer and a final **Dense** layer with Sigmoid activation function for binary classification. Additional, you may want to add **Dropout** regularisation to your model.

You may notice that the **Convolutional** layer (Conv2D) has fixed hyperparameters' values (number of filters, kernel size, etc.). Change the code in a way that Conv2D can use the values listed in the definition of the *create_model* method.

In [7]:
# Function to create the CNN model
def create_model(optimizer='adam', filters = 100, kernel_size=(3,3), strides=(1,1), padding='same',learning_rate = 0.001,dropout_rate=0.1):
    cnn_model = Sequential(name  = "cnn")
    cnn_model.add(Conv2D(filters=filters, kernel_size=(3, X_train.shape[2]), input_shape=X_train.shape[1:], data_format='channels_last', activation='relu', padding=padding, strides=strides))
    cnn_model.add(Dropout(dropout_rate))
    cnn_model.add(GlobalMaxPooling2D())
    cnn_model.add(Flatten())
    cnn_model.add(Dense(1, activation='sigmoid'))

    compileModel(cnn_model, optimizer,learning_rate)
    print (cnn_model.summary())
    return cnn_model


# Random search
The code in the following cell implements *random search* to tune the hyperparameters of the CNN model. 

In the cell below, add the relevant hyperparameters for the CNN to the **param_dist** dictionary. The tunable hyperparameters are those available in definition of the *create_model* above. Remember to use the *uniform* method for generating floating point values (e.g., for the **learning_rate**), use *randint* for generating the integer hyperparameters (e.g., the **number of filters**), while use lists for multi-dimensional hyperparameters (e.g., **kernel_size**). 

In [9]:
from scipy.stats import uniform, randint
k=2 # number of folds for cross-validation
PATIENCE = 10

# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)

# Define the hyperparameters to tune and their possible values
param_dist = {
    'learning_rate' : uniform(0.0001, 0.001),
    'filters' : randint(16,64),
    'kernel_size': [(2,2),(3,3),(2,3)],
    'strides': [(1,1),(2,2)],
    'padding' : ['same', 'valid']
}

# Perform grid search with 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=5, cv=k, random_state=SEED)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=PATIENCE, restore_best_weights=True)
start_time = time.time()
random_search_result = random_search.fit(X_train, y_train,epochs=100, validation_data=(X_val, y_val),callbacks= [ early_stopping])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)

# Print the best parameters and corresponding accuracy
print("Best parameters found: ", random_search_result.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(random_search_result.best_score_))

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)

# F1 score on the best model on test set
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print("F1 score of the best model: {:.2f}".format(f1))
print("Test accuracy of the best model: {:.2f}".format(test_accuracy))
# Print the false positives and false negatives
# print(classification_report(y_test, y_pred))

# Save the results in a log file
with open("random_search_log.txt", "a") as f:
    f.write("Shape of the train dataset: {}\n".format(X_train.shape)) 
    f.write("Shape of the test dataset: {}\n".format(X_test.shape)) 
    f.write("Shape of the validation dataset: {}\n".format(X_val.shape)) 
    f.write("Best parameters found: {}\n".format(random_search_result.best_params_))
    f.write("Best cross-validated accuracy: {:.2f}\n".format(random_search_result.best_score_))
    f.write("F1 score of the best model: {:.2f}\n".format(f1))
    f.write("Test accuracy of the best model: {:.2f}\n".format(test_accuracy))
    f.write("\n\n")
    

/tmp/ipykernel_25240/424657782.py:6: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)


Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 2, 2, 55)          385       
                                                                 
 dropout_15 (Dropout)        (None, 2, 2, 55)          0         
                                                                 
 global_max_pooling2d_15 (Gl  (None, 55)               0         
 obalMaxPooling2D)                                               
                                                                 
 flatten_15 (Flatten)        (None, 55)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 56        
                                                                 
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________

/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
2 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/keras/wrappers/scikit_learn.py", line 232, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/home/alessia/miniconda3/envs/NIADML/lib/python3.9/site-packages/keras/wrappe

15/15 [==============================] - 1s 14ms/step - loss: 0.7104 - accuracy: 0.4039 - val_loss: 0.6497 - val_accuracy: 0.5901
Epoch 2/100
15/15 [==============================] - 0s 4ms/step - loss: 0.6556 - accuracy: 0.5961 - val_loss: 0.6040 - val_accuracy: 0.9006
Epoch 3/100
15/15 [==============================] - 0s 5ms/step - loss: 0.6195 - accuracy: 0.8658 - val_loss: 0.5658 - val_accuracy: 0.9752
Epoch 4/100
15/15 [==============================] - 0s 5ms/step - loss: 0.5804 - accuracy: 0.9302 - val_loss: 0.5327 - val_accuracy: 0.9814
Epoch 5/100
15/15 [==============================] - 0s 5ms/step - loss: 0.5386 - accuracy: 0.9405 - val_loss: 0.5014 - val_accuracy: 0.9814
Epoch 6/100
15/15 [==============================] - 0s 5ms/step - loss: 0.5070 - accuracy: 0.9523 - val_loss: 0.4704 - val_accuracy: 0.9876
Epoch 7/100
15/15 [==============================] - 0s 5ms/step - loss: 0.4770 - accuracy: 0.9606 - val_loss: 0.4415 - val_accuracy: 0.9876
Epoch 8/100
15/15 [=====

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Reduce the dataset for simplicity
X_train_reduced = X_train.reshape(X_train.shape[0], -1)[:1000]
y_train_reduced = y_train[:1000]
X_test_reduced = X_test.reshape(X_test.shape[0], -1)[:200]
y_test_reduced = y_test[:200]

# Create and train the SVM model
svm_model = SVC(kernel='linear', random_state=SEED)
svm_model.fit(X_train_reduced, y_train_reduced)

# Predict on the test set
y_pred_svm = svm_model.predict(X_test_reduced)

# Calculate accuracy
accuracy = accuracy_score(y_test_reduced, y_pred_svm)
print(f"SVM Test Accuracy: {accuracy:.2f}")

SVM Test Accuracy: 0.98
